In [1]:
import json
import pandas as pd
import numpy as np
import sys

pd.set_option('display.max_colwidth', None)
sys.path.append('./src-py')

In [2]:
import sbert_training
from utils import *

In [3]:
from datasets import load_dataset, load_metric, Dataset, Split
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline, DebertaForSequenceClassification
from transformers import TrainingArguments, Trainer
import wandb
import torch
from tqdm import tqdm

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=123)

In [4]:
output_path = "../../data-ceph/arguana/argmining22-sharedtask/models/"

In [5]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taska_valid_df    = pd.read_csv('../data/TaskA_dev.csv')
taska_test_df     = pd.read_csv('../data/TaskA_test.csv') #Replace this with the path to the test file

taska_training_df.insert(loc=0,column='row_num',value=np.arange(len(taska_training_df)))
taska_valid_df.insert(loc=0,column='row_num',value=np.arange(len(taska_valid_df)))
taska_test_df.insert(loc=0,column='row_num',value=np.arange(len(taska_test_df)))

#The </s></s>  is the separator used in the pre-trained nli model..
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '{}:{}  </s></s> {} '.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
taska_valid_df['input_txt']    = taska_valid_df.apply(lambda x: '{}:{} </s></s> {}'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)
taska_test_df['input_txt']     = taska_test_df.apply(lambda x: '{}:{} </s></s> {}'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)

taska_validity_train_df = taska_training_df[taska_training_df.Validity != 0].copy()
taska_validity_valid_df = taska_valid_df[taska_valid_df.Validity != 0].copy()
taska_validity_test_df  = taska_test_df[taska_test_df.Validity != 0].copy()

taska_validity_train_df['label'] = taska_validity_train_df.Validity.apply(lambda x : "valid" if x == 1 else "invalid")
taska_validity_valid_df['label'] = taska_validity_valid_df.Validity.apply(lambda x : "valid" if x == 1 else "invalid")
taska_validity_test_df['label']  = taska_validity_test_df.Validity.apply(lambda x  : "valid" if x == 1 else "invalid")


taska_novelty_train_df = taska_training_df[taska_training_df.Novelty != 0].copy()
taska_novelty_valid_df = taska_valid_df[taska_valid_df.Novelty != 0].copy()
taska_novelty_test_df  = taska_test_df[taska_test_df.Novelty != 0].copy()

#Balancing the data for novelty task..
taska_novelty_train_balanced_df, y = ros.fit_resample(taska_novelty_train_df, taska_novelty_train_df['Novelty'])
taska_novelty_train_balanced_df['Novelty'] = y

taska_novelty_train_df['label'] = taska_novelty_train_df.Novelty.apply(lambda x : "novel" if x == 1 else "conservative")
taska_novelty_train_balanced_df['label'] = taska_novelty_train_balanced_df.Novelty.apply(lambda x : "novel" if x == 1 else "conservative")
taska_novelty_valid_df['label'] = taska_novelty_valid_df.Novelty.apply(lambda x : "novel" if x == 1 else "conservative")
taska_novelty_test_df['label']  = taska_novelty_test_df.Novelty.apply(lambda x  : "novel" if x == 1 else "conservative")

## Fine-tune simple RoBERTa model on the training data for Novelty:

In [6]:
bert_tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [7]:
taska_novelty_train_df.columns

Index(['row_num', 'topic', 'Premise', 'Conclusion', 'Validity',
       'Validity-Confidence', 'Novelty', 'Novelty-Confidence', 'input_txt',
       'label'],
      dtype='object')

In [8]:
taska_novelty_train_df.Novelty.value_counts()

-1    595
 1    123
Name: Novelty, dtype: int64

In [9]:
validity_map = dict([ # avoid negative labels
    ("novel", 1), 
    ("conservative", 0)
])

In [10]:
#train_dataset = Dataset.from_pandas(taska_novelty_train_df)
train_dataset = Dataset.from_pandas(taska_novelty_train_balanced_df)
eval_dataset = Dataset.from_pandas(taska_novelty_valid_df)
test_dataset = Dataset.from_pandas(taska_novelty_test_df)

In [11]:
def preprocess(example):
    inputs = bert_tokenizer(example["input_txt"], add_special_tokens=False, padding=True, truncation=True, max_length=512)
    inputs['label'] = list(map(validity_map.get, example['label']))
    return inputs

In [12]:
train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

2022-08-23 16:56:36 - Parameter 'function'=<function preprocess at 0x7f54ddf91f70> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
taska_novelty_train_balanced_df.Novelty.value_counts()

 1    595
-1    595
Name: Novelty, dtype: int64

In [14]:
bert_model     = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

training_args = TrainingArguments(
    output_dir= output_path + "/novelty/roberta", 
    #report_to="wandb",
    logging_dir='/var/argmining-sharedtask/roberta-baseline-novelty',
    overwrite_output_dir=True,
    metric_for_best_model = 'f1',
    evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
    learning_rate = 5e-6,                   # we can customize learning rate
    max_steps = 600,
    logging_steps = 50,                    # we will log every 50 steps which is an epoch given the 700 examples and 16 batch size
    eval_steps = 50,                      # we will perform evaluation every 500 steps
    save_steps = 50,
    load_best_model_at_end = True,
)

trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda x: compute_metrics(x, average='macro')
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [15]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: topic, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty. If topic, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1190
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total op

Step,Training Loss,Validation Loss,Recall,Precision,F1
50,0.697800,0.705404,0.500000,0.205000,0.290780
100,0.698100,0.697526,0.484601,0.428331,0.319202
150,0.693100,0.697641,0.465482,0.426172,0.348958
200,0.679100,0.692525,0.500000,0.295000,0.371069
250,0.669500,0.726181,0.437784,0.346272,0.363306
300,0.627800,0.776755,0.343634,0.347955,0.344762
350,0.597900,0.806083,0.422695,0.344684,0.360502
400,0.544800,0.816149,0.423005,0.393190,0.394306
450,0.515600,0.863293,0.417941,0.359320,0.368626
500,0.461500,0.905719,0.428793,0.361061,0.370305


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty. If topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ../../data-ceph/arguana/argmining22-sharedtask/models//novelty/roberta/checkpoint-50
C

TrainOutput(global_step=600, training_loss=0.5922914218902587, metrics={'train_runtime': 67.715, 'train_samples_per_second': 70.885, 'train_steps_per_second': 8.861, 'total_flos': 305356824186240.0, 'train_loss': 0.5922914218902587, 'epoch': 4.03})

In [16]:
trainer.evaluate(test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty, Topic-in-dev-split. If topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty, Topic-in-dev-split are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 520
  Batch size = 8


{'eval_loss': 0.7623319625854492,
 'eval_recall': 0.5196857504063572,
 'eval_precision': 0.5211568322981367,
 'eval_f1': 0.5168408826945412,
 'eval_runtime': 0.6987,
 'eval_samples_per_second': 744.215,
 'eval_steps_per_second': 93.027,
 'epoch': 4.03}

In [17]:
results = trainer.predict(test_dataset)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty, Topic-in-dev-split. If topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty, Topic-in-dev-split are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 520
  Batch size = 8


In [19]:
taska_test_df['predicted novelty']  = [1 if np.argmax(x) == 1 else -1 for x in results.predictions]

## Fine-tune simple RoBERTa model on the training data for Validity:

In [20]:
train_dataset = Dataset.from_pandas(taska_validity_train_df)
eval_dataset = Dataset.from_pandas(taska_validity_valid_df)
test_dataset = Dataset.from_pandas(taska_validity_test_df)

In [21]:
validity_map = dict([ # avoid negative labels
    ("valid", 1), 
    ("invalid", 0)
])

train_dataset = train_dataset.map(preprocess, batched=True)
eval_dataset = eval_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
taska_validity_train_df.label.value_counts()

valid      401
invalid    320
Name: label, dtype: int64

In [23]:
bert_model     = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

training_args = TrainingArguments(
    output_dir= output_path + "/validity/roberta", 
    #report_to="wandb",
    logging_dir='/var/argmining-sharedtask/roberta-baseline-validity',
    overwrite_output_dir=True,
    metric_for_best_model = 'f1',
    evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
    learning_rate = 5e-6,                   # we can customize learning rate
    max_steps = 600,
    logging_steps = 50,                    # we will log every 50 steps which is an epoch given the 700 examples and 16 batch size
    eval_steps = 50,                      # we will perform evaluation every 500 steps
    save_steps = 50,
    load_best_model_at_end = True,
)

trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda x: compute_metrics(x, average='macro')
)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /mnt/ceph/storage/data-tmp/current//sile2804/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights fil

In [24]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty. If topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 721
  Num Epochs = 7
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradi

Step,Training Loss,Validation Loss,Recall,Precision,F1
50,0.683800,0.667877,0.500000,0.314070,0.385802
100,0.691200,0.673957,0.500000,0.314070,0.385802
150,0.694300,0.675512,0.500000,0.314070,0.385802
200,0.686300,0.666249,0.500000,0.314070,0.385802
250,0.667600,0.631072,0.500000,0.314070,0.385802
300,0.609800,0.561435,0.659946,0.741827,0.664523
350,0.461200,0.615632,0.661189,0.759756,0.665185
400,0.483000,0.590081,0.670703,0.758091,0.676948
450,0.396400,0.614422,0.674703,0.769603,0.681420
500,0.449100,0.650877,0.638162,0.752071,0.635531


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty. If topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 199
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ../../data-ceph/arguana/argmining22-sharedtask/models//validity/roberta/checkpoint-50


TrainOutput(global_step=600, training_loss=0.5532616901397706, metrics={'train_runtime': 56.2433, 'train_samples_per_second': 85.343, 'train_steps_per_second': 10.668, 'total_flos': 303190269089760.0, 'train_loss': 0.5532616901397706, 'epoch': 6.59})

In [25]:
trainer.evaluate(test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty, Topic-in-dev-split. If topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty, Topic-in-dev-split are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 520
  Batch size = 8


{'eval_loss': 0.7923006415367126,
 'eval_recall': 0.6069661740152125,
 'eval_precision': 0.6565703423774072,
 'eval_f1': 0.6000896502322556,
 'eval_runtime': 0.7187,
 'eval_samples_per_second': 723.568,
 'eval_steps_per_second': 90.446,
 'epoch': 6.59}

In [26]:
results = trainer.predict(test_dataset)
taska_test_df['predicted validity']  = [1 if np.argmax(x) == 1 else -1 for x in results.label_ids]

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty, Topic-in-dev-split. If topic, __index_level_0__, input_txt, Conclusion, Validity-Confidence, Novelty-Confidence, Validity, row_num, Premise, Novelty, Topic-in-dev-split are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 520
  Batch size = 8


In [27]:
#taska_test_df[['row_num', 'topic', 'Premise', 'Conclusion', 'predicted validity', 'predicted novelty']].to_csv('../data/output/roberta_single_predictions.csv')
taska_test_df[['row_num', 'topic', 'Premise', 'Conclusion', 'predicted validity', 'predicted novelty']].to_csv('../data/output/balanced_roberta_single_predictions.csv')